<a href="https://colab.research.google.com/github/kajalpani/PySpark_work/blob/main/spark_dataframe_covid_positivity_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [21]:
%%sh
unzip us-counties.csv.zip

Archive:  us-counties.csv.zip
  inflating: us-counties.csv         


In [22]:
%%sh
head us-counties.csv

date,county,state,fips,cases,deaths
2020-01-21,Snohomish,Washington,53061,1,0
2020-01-22,Snohomish,Washington,53061,1,0
2020-01-23,Snohomish,Washington,53061,1,0
2020-01-24,Cook,Illinois,17031,1,0
2020-01-24,Snohomish,Washington,53061,1,0
2020-01-25,Orange,California,06059,1,0
2020-01-25,Cook,Illinois,17031,1,0
2020-01-25,Snohomish,Washington,53061,1,0
2020-01-26,Maricopa,Arizona,04013,1,0


In [24]:
covid_df = spark.read.csv('us-counties.csv', header=True, inferSchema=True)
covid_df

date,county,state,fips,cases,deaths
2020-01-21,Snohomish,Washington,53061,1,0
2020-01-22,Snohomish,Washington,53061,1,0
2020-01-23,Snohomish,Washington,53061,1,0
2020-01-24,Cook,Illinois,17031,1,0
2020-01-24,Snohomish,Washington,53061,1,0
2020-01-25,Orange,California,6059,1,0
2020-01-25,Cook,Illinois,17031,1,0
2020-01-25,Snohomish,Washington,53061,1,0
2020-01-26,Maricopa,Arizona,4013,1,0
2020-01-26,Los Angeles,California,6037,1,0


In [32]:
covid_df.filter(covid_df.cases > 1).limit(5)

date,county,state,fips,cases,deaths
2020-01-30,Cook,Illinois,17031,2,0
2020-01-31,Cook,Illinois,17031,2,0
2020-02-01,Cook,Illinois,17031,2,0
2020-02-02,San Francisco,California,6075,2,0
2020-02-02,Santa Clara,California,6085,2,0


Find out which US State had maximum number of cases on 2020

In [51]:
covid_df.createOrReplaceTempView('covid_us')
spark.sql('select count(cases), state from covid_us where cases > 1  and substring(date,1,4) == "2020" group by state order by count(cases) DESC')

count(cases),state
64449,Texas
44618,Georgia
36260,Virginia
31698,Kentucky
30275,Missouri
27637,North Carolina
27295,Illinois
26476,Tennessee
26362,Iowa
25654,Indiana
